<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d438677e8402ff4de69302b9739af1aebfd1c48a222217946e7a866989df7b8a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-28 16:21:36
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -30.37 K (-0.21%)
Current PnL: -22.67 L (-14.86%)
CY Booked + Current PnL: -8.78 L (-5.75%)
-------------------
Total profit:  1.66 L
Total loss:  -24.33 L
-------------------
Total Booked + Current PnL: 18.20 L (14.49%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.71%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.08 L (61.54%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.0,0.22,-8.81,9.69,0.02,15424.0,-15386.0,159173.0,142.75,67.0,M-SC,10.45,234.0,-1.00,1.11,38.45,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.0,-2.57,-16.33,19.62,0.09,16545.0,-16452.0,84325.0,95.73,38.0,M-SC,3.50,253.0,-0.99,0.59,10.78,OX40N,NTT,DURABLES
83,VOLTAS,1530.0,-1.55,7.67,11.17,19.69,23060.0,14703.0,206445.0,-2.87,50.0,X-MC,3.32,78.0,0.64,1.44,14.99,XY25,NTT,AC
43,ITC,452.0,-0.01,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,46.0,X-LC,1.20,5.0,-0.11,1.38,4.27,X40,NTT,FMCG
52,MEDANTA,1486.0,-0.32,4.04,19.26,24.08,25438.0,5126.0,132076.0,-4.36,47.0,X-SC,5.88,89.0,0.20,0.92,24.45,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,3.05,-2.73,39.47,35.67,121263.0,-8613.0,307229.0,-14.36,62.0,X-LC,5.72,4.0,-0.07,2.15,10.75,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,2.16,0.35,17.02,17.44,25594.0,528.0,150375.0,-13.50,55.0,X-MC,4.12,63.0,0.02,1.05,19.58,X40,ATH,PHARMA
27,GLAXO,3466.20,1.98,3.15,34.90,39.15,71740.0,6276.0,205560.0,-17.11,50.0,X-MC,10.17,60.0,0.09,1.44,33.06,X40,ATH,PHARMA
14,CAMPUS,393.00,1.73,-6.43,42.44,33.28,66625.0,-10788.0,156987.0,-24.66,53.0,M-SC,2.93,222.0,-0.16,1.10,21.68,XY24,NTT,FOOTWEAR
51,MASFIN,398.61,1.45,-5.45,29.08,22.05,26940.0,-5340.0,92640.0,-18.70,50.0,H-SC,6.99,164.0,-0.20,0.65,34.66,XR,ATH,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIS,528.00,-4.50,-24.38,60.36,21.26,50678.0,-27073.0,83959.0,1980.31,46.0,H-SC,3.89,166.0,-0.53,0.59,13.44,OX40N,NTT,MISC
58,RAJOOENG,143.10,-3.78,-27.66,92.99,39.61,68952.0,-28350.0,74150.0,-51.04,35.0,H-SC,21.15,135.0,-0.41,0.52,4.51,AR,ATH,MISC
72,TANLA,1943.92,-2.58,-37.39,250.89,119.69,425319.0,-101237.0,169524.0,-36.76,29.0,H-SC,10.31,156.0,-0.24,1.18,33.92,AR,ATH,IT
78,TTKPRESTIG,770.00,-2.57,-16.33,19.62,0.09,16545.0,-16452.0,84325.0,95.73,38.0,M-SC,3.50,253.0,-0.99,0.59,10.78,OX40N,NTT,DURABLES
44,JCHAC,2282.00,-2.35,-29.27,41.36,-0.01,37390.0,-37404.0,90401.0,18400.00,12.0,M-SC,1.17,233.0,-1.00,0.63,2.37,OX40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.56,-0.73,108.25,106.72,132530.0,-906.0,122430.0,-50.82,36.0,H-SC,2.74,139.0,-0.01,0.86,22.47,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.56,-0.61,36.46,35.63,75943.0,-1282.0,208292.0,-14.53,44.0,H-MC,3.78,119.0,-0.02,1.46,15.96,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.60,-5.12,119.12,107.91,166876.0,-7553.0,140091.0,-22.98,35.0,M-SC,10.68,216.0,-0.05,0.98,0.17,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.22,-8.81,9.69,0.02,15424.0,-15386.0,159173.0,142.75,67.0,M-SC,10.45,234.0,-1.00,1.11,38.45,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-2.57,-16.33,19.62,0.09,16545.0,-16452.0,84325.0,95.73,38.0,M-SC,3.50,253.0,-0.99,0.59,10.78,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.10,-21.07,43.77,13.47,93161.0,-56826.0,212841.0,-67.53,39.0,H-SC,1.60,158.0,-0.61,1.49,8.22,XY24,NTT,PAINTS
67,SIS,528.00,-4.50,-24.38,60.36,21.26,50678.0,-27073.0,83959.0,1980.31,46.0,H-SC,3.89,166.0,-0.53,0.59,13.44,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.01,2.35,68.32,72.27,118483.0,3975.0,173423.0,-11.98,59.0,M-LC,3.69,99.0,0.03,1.21,8.67,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.01,2.35,68.32,72.27,118483.0,3975.0,173423.0,-11.98,59.0,M-LC,3.69,99.0,0.03,1.21,8.67,XR,NTT,IT
38,INDIAMART,4810.62,-0.56,-0.73,108.25,106.72,132530.0,-906.0,122430.0,-50.82,36.0,H-SC,2.74,139.0,-0.01,0.86,22.47,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.56,-0.61,36.46,35.63,75943.0,-1282.0,208292.0,-14.53,44.0,H-MC,3.78,119.0,-0.02,1.46,15.96,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.60,-5.12,119.12,107.91,166876.0,-7553.0,140091.0,-22.98,35.0,M-SC,10.68,216.0,-0.05,0.98,0.17,OX40N,ATH,CABLES
37,IEX,219.00,-1.14,-5.47,57.23,48.63,108174.0,-10937.0,189017.0,-37.19,48.0,H-SC,15.61,136.0,-0.10,1.32,5.45,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.09,-35.55,109.77,35.20,91037.0,-45736.0,82934.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.28,-15.45,68.16,42.18,159293.0,-42706.0,233704.0,-24.30,28.0,X-LC,1.95,3.0,-0.27,1.63,1.23,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.19,-8.91,26.50,15.23,48359.0,-17861.0,182486.0,-22.11,29.0,X-MC,6.97,66.0,-0.37,1.28,20.46,X40N,NTT,REALTY
9,BAJAJHFL,181.50,-0.27,-14.06,72.74,48.45,125342.0,-28191.0,172315.0,-20.30,31.0,X-MC,7.54,64.0,-0.22,1.20,0.10,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.19,-4.72,40.39,33.76,77282.0,-9480.0,191340.0,-32.78,32.0,X-MC,3.89,57.0,-0.12,1.34,0.23,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.01,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,46.0,X-LC,1.20,5.0,-0.11,1.38,4.27,X40,NTT,FMCG
3,ACC,3906.0,-1.05,-22.23,111.07,64.15,205546.0,-52891.0,185060.0,-54.57,49.0,X-SC,1.57,82.0,-0.26,1.29,3.81,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.62,1.42,23.04,24.79,58853.0,3584.0,255440.0,-3.25,48.0,X-SC,1.86,86.0,0.06,1.78,24.89,X40N,NTT,MISC
66,SIEMENS,4671.5,-0.46,-11.42,41.69,25.51,68726.0,-21245.0,164850.0,1.14,64.0,H-LC,1.87,49.0,-0.31,1.15,21.48,AR,ATH,ELECTRICAL
76,TMPV,600.0,-0.28,-15.45,68.16,42.18,159293.0,-42706.0,233704.0,-24.30,28.0,X-LC,1.95,3.0,-0.27,1.63,1.23,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.09,-35.55,109.77,35.20,91037.0,-45736.0,82934.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-1.57,-6.39,34.67,26.07,53142.0,-10460.0,153280.0,-31.05,34.0,X-MC,9.58,55.0,-0.20,1.07,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.27,-14.06,72.74,48.45,125342.0,-28191.0,172315.0,-20.30,31.0,X-MC,7.54,64.0,-0.22,1.20,0.10,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.19,-4.72,40.39,33.76,77282.0,-9480.0,191340.0,-32.78,32.0,X-MC,3.89,57.0,-0.12,1.34,0.23,X40,ATH,FMCG
76,TMPV,600.00,-0.28,-15.45,68.16,42.18,159293.0,-42706.0,233704.0,-24.30,28.0,X-LC,1.95,3.0,-0.27,1.63,1.23,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.71,-27.50,97.03,42.84,45729.0,-17877.0,47129.0,-52.62,42.0,X-SC,37.56,83.0,-0.39,0.33,2.55,XY24,NTT,MISC
60,RELAXO,1176.00,-0.22,-47.00,187.35,52.31,144145.0,-68221.0,76939.0,-43.57,37.0,X-SC,4.52,91.0,-0.47,0.54,2.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.09,-35.55,109.77,35.20,91037.0,-45736.0,82934.0,8.74,24.0,X-SC,13.35,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.29,-6.23,29.66,21.58,38976.0,-8730.0,131409.0,-52.72,36.0,X-MC,6.42,56.0,-0.22,0.92,16.34,X40N,ATH,IT
52,MEDANTA,1486.00,-0.32,4.04,19.26,24.08,25438.0,5126.0,132076.0,-4.36,47.0,X-SC,5.88,89.0,0.20,0.92,24.45,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.03,-14.75,39.92,19.28,117734.0,-51031.0,294925.0,-25.57,58.0,X-LC,7.11,1.0,-0.43,2.06,9.03,X40,ATH,IT
41,INFY,2275.00,-0.40,6.20,45.82,54.87,152260.0,19413.0,332301.0,-17.44,61.0,X-LC,2.48,2.0,0.13,2.32,15.00,X40,BTT,IT
76,TMPV,600.00,-0.28,-15.45,68.16,42.18,159293.0,-42706.0,233704.0,-24.30,28.0,X-LC,1.95,3.0,-0.27,1.63,1.23,XY24,NTT,AUTO
82,VBL,671.64,3.05,-2.73,39.47,35.67,121263.0,-8613.0,307229.0,-14.36,62.0,X-LC,5.72,4.0,-0.07,2.15,10.75,X40N,ATH,FMCG
43,ITC,452.00,-0.01,-1.23,11.81,10.44,23346.0,-2460.0,197678.0,-41.15,46.0,X-LC,1.20,5.0,-0.11,1.38,4.27,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.13,-36.62,109.66,32.89,53664.0,-28270.0,48937.0,-704.67,65.0,L-MC,5.76,259.0,-0.53,0.34,34.77,XR,NTT,BANKS
6,ASIANTILES,137.00,-1.98,-13.77,110.67,81.67,89529.0,-12913.0,80897.0,7272.22,48.0,L-SC,19.50,271.0,-0.14,0.57,56.66,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.44,-11.47,57.62,39.54,51602.0,-11603.0,89556.0,-30.76,68.0,M-SC,6.68,220.0,-0.22,0.63,26.28,AR,ATH,AUTO
51,MASFIN,398.61,1.45,-5.45,29.08,22.05,26940.0,-5340.0,92640.0,-18.70,50.0,H-SC,6.99,164.0,-0.20,0.65,34.66,XR,ATH,FINANCE
70,SURYODAY,216.00,0.48,-19.51,52.86,23.03,76190.0,-34935.0,144136.0,57.89,49.0,H-SC,9.58,167.0,-0.46,1.01,43.01,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.22,-8.81,9.69,0.02,15424.0,-15386.0,159173.0,142.75,67.0,M-SC,10.45,234.0,-1.00,1.11,38.45,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.16,0.47,20.38,20.95,51551.0,1179.0,252947.0,-0.49,74.0,X-LC,15.17,31.0,0.02,1.77,36.27,X40,ATH,PAINTS
50,LTIM,7201.88,1.18,11.45,18.13,31.66,49738.0,28191.0,274342.0,7.16,67.0,H-LC,13.59,43.0,0.57,1.92,52.22,X200,ATH,IT
79,UNITDSPR,1644.00,0.40,6.63,13.25,20.77,33082.0,15531.0,249675.0,-2.63,62.0,X-MC,3.78,62.0,0.47,1.74,14.01,X40N,NTT,BREWERIES
40,INDUSINDBK,1800.00,0.13,-36.62,109.66,32.89,53664.0,-28270.0,48937.0,-704.67,65.0,L-MC,5.76,259.0,-0.53,0.34,34.77,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.94
1,25,44.85
2,50,75.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.40
MC    29.24
LC    25.36
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.80
X40      21.28
X40N     12.00
XR        9.53
XY25      9.10
AR        9.06
OX40N     7.58
X200      1.92
SR        0.97
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.84
X-MC    22.75
X-LC    20.11
M-SC    11.91
X-SC     8.25
H-MC     4.76
H-LC     3.07
L-SC     1.40
M-MC     1.39
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.87,-4.28,38.00
IT,13.78,-14.59,73.75
FINANCE,9.49,-14.03,62.21
MISC,7.33,-23.88,74.64
ELECTRICAL,6.00,-9.44,49.52
PAINTS,5.97,-8.17,24.23
INSURANCE,4.40,-0.30,35.65
PHARMA,3.95,0.98,30.71
AUTO,3.38,-18.72,69.62


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3054508.0,21
XR,1299184.0,13
AR,1277387.0,10
X40,993973.0,14
X40N,731602.0,9
OX40N,706911.0,10
XY25,338539.0,6
SR,280835.0,2
MH,75414.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3565601.0,25
M-SC,1354291.0,15
X-MC,1228369.0,16
X-LC,884098.0,11
X-SC,738250.0,8
H-MC,398202.0,3
L-SC,261656.0,3
M-LC,118483.0,1
H-LC,118464.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1210573.0      6
           AR         879925.0      5
           XR         794209.0      7
M-SC       XY24       780482.0      6
X-MC       X40        435619.0      7
X-LC       X40        391658.0      5
X-MC       XY24       368117.0      3
H-SC       OX40N      324645.0      4
X-SC       X40N       294841.0      3
M-SC       OX40N      294429.0      5
H-SC       SR         280835.0      2
X-SC       XY24       276713.0      3
X-MC       X40N       245759.0      4
X-LC       XY24       228786.0      2
H-MC       AR         208365.0      2
X-LC       X40N       191002.0      2
H-MC       XY24       189837.0      1
X-MC       XY25       178874.0      2
L-SC       XR         173819.0      2
X-SC       X40        166696.0      2
M-SC       XR         159009.0      2
           AR         120371.0      2
M-LC       XR         118483.0      1
L-SC       OX40N       87837.0      1
H-SC       MH          75414.0      1
X-LC       XY25        72652.0      2
H-LC       AR          68726.0      1
L-MC       XR          53664.0      1
H-LC       X200        49738.0      1
M-MC       XY25        46805.0      1
L-LC       XY25        40208.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
